In [2]:
import requests
import pandas as pd
import numpy as np

JSON object is extracted from the Bootstrap static API which contains static data after the end of a certain gameweek.

You can find more about the kinds of APIs available in Frenzel's article, <br>
(https://medium.com/@frenzelts/fantasy-premier-league-api-endpoints-a-detailed-guide-acbd5598eb19)

In [3]:
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()
json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [14]:
elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])

#### Elements Dataframe 

In [6]:
#elements_df['web_name'].unique()
#elements_df['web_name'].unique().shape[0]/20

The above has the list of entire players from the dataset

#### Merging the dataframes

In [7]:
playerdata = elements_df.copy()

Adding the player's position from the element type dataframe

In [10]:
playerdata['position'] = playerdata.element_type.map(elements_types_df.set_index('id').singular_name)

In [11]:
playerdata.head(5)

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,position
0,NaN,NaN,80201,0,0,-2,2,0,1,0.5,...,52,287,20,NaN,,NaN,,NaN,,Goalkeeper
1,0.0,0.0,115918,0,0,0,0,0,1,0.0,...,14,425,37,NaN,,NaN,,NaN,,Goalkeeper
2,0.0,0.0,47431,0,0,-2,2,0,3,0.0,...,248,607,248,NaN,,NaN,,NaN,,Midfielder
3,100.0,100.0,54694,0,0,-1,1,0,4,5.5,...,6,40,11,NaN,,NaN,,1.0,,Forward
4,NaN,NaN,58822,0,0,-2,2,0,2,0.7,...,118,304,105,NaN,,NaN,,NaN,,Defender


In [12]:
playerdata.columns.shape

(68,)

Adding the player's team

In [15]:
playerdata['team'] = playerdata.team.map(teams_df.set_index('id').name)

Converting the value_season columns to float type

In [16]:
playerdata['value'] = playerdata.value_season.astype(float)

In [17]:
playerdata.sort_values('value',ascending=False).head(10)

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,position,value
69,NaN,NaN,61810,0,0,1,-1,0,2,3.0,...,181,57,NaN,,NaN,,NaN,,Defender,8.3
84,100.0,100.0,231065,0,0,1,-1,1,2,3.8,...,110,28,NaN,,NaN,,NaN,,Defender,8.3
102,NaN,NaN,61933,0,0,3,-3,1,2,5.0,...,51,7,NaN,,NaN,,NaN,,Defender,8.1
242,NaN,NaN,60252,1,-1,1,-1,3,3,8.5,...,20,13,1.0,,2.0,,4.0,,Midfielder,7.7
232,NaN,NaN,121599,1,-1,2,-2,1,3,7.0,...,23,16,NaN,,NaN,,NaN,,Midfielder,7.5
354,NaN,NaN,121145,1,-1,2,-2,2,2,9.0,...,46,5,NaN,,NaN,,NaN,,Defender,7.1
569,NaN,NaN,172841,0,0,6,-6,2,3,4.5,...,11,6,NaN,,2.0,,NaN,,Midfielder,6.7
187,NaN,NaN,180736,0,0,-2,2,2,2,5.2,...,170,51,NaN,,NaN,,NaN,,Defender,6.7
165,NaN,NaN,82263,0,0,5,-5,2,2,5.0,...,29,2,2.0,,2.0,,NaN,,Defender,6.5
73,NaN,NaN,154561,0,0,1,-1,0,1,2.2,...,194,8,NaN,,NaN,,NaN,,Goalkeeper,6.5


In [18]:
#playerdata.shape

In [19]:
pivot=playerdata.pivot_table(index='position',values='value',aggfunc=np.mean).reset_index()

pivot.sort_values('value',ascending=False)

,position,value
0,Defender,1.890385
3,Midfielder,1.640079
2,Goalkeeper,1.488235
1,Forward,1.388235


In [20]:
team_pivot = playerdata.pivot_table(index='team',values='value',aggfunc=np.mean).reset_index()

team_pivot.sort_values('value',ascending=False)

,team,value
5,Chelsea,2.316667
2,Brentford,2.203333
11,Man City,2.164286
10,Liverpool,2.086667
18,West Ham,1.881481
12,Man Utd,1.807407
3,Brighton,1.802941
1,Aston Villa,1.787879
7,Everton,1.785294
6,Crystal Palace,1.733333


In [21]:
playerdata.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'influence_rank', 'influence_rank_type', 'creativity_rank',
       'creativity_rank_type', 'threat_rank'

In [23]:
#playerdata['ep_next'].unique()

In [24]:
playerdata.to_excel('FPL_backend_v10102021_0900.xlsx')